# AI Game Referee: Rock-Paper-Scissors-Plus

## My Approach

For this assignment, I wanted to build more than just a chatbot wrapper. My primary goal was to solve the common "LLM Hallucination" problem. Since LLMs are probabilistic, they often lose track of scores or invent rules in long conversations.

To fix this, I treated the assignment as a **Systems Engineering** problem. I separated the "Brain" (Gemini) from the "Rulebook" (Python), ensuring the game is mathematically fair and deterministic while still feeling conversational.

## 1. How I Managed Game State

The requirements stated that the game state must not live only in the prompt. I took this strictly.

* **The Single Source of Truth:** I created a Pydantic-style `GameState` class that lives in the Python runtime, not in the chat history.
* **What it Tracks:** It explicitly holds the `round_count`, numeric scores, and crucially boolean flags for `user_bomb_used`.
* **Why I did this:** This ensures that even if the LLM "forgets" I used a bomb in Round 1, the Python object remembers. The LLM simply reads this immutable state every turn.

## 2. The "Blind Referee" Architecture

I designed the agent to act as a **Interface Layer**, not a Judge.

* **Intent Understanding:** The Gemini Agent's only job is to understand what the user wants to do (e.g., "I throw a rock").
* **The Logic Engine (Tool):** I built a deterministic tool called `resolve_round`. The Agent is forced via system instructions to pass every move to this tool.
* **The flow:** User Input $\rightarrow$ Agent parses intent $\rightarrow$ Tool calculates winner $\rightarrow$ Agent reports result.

## 3. Trade-offs & Challenges

* **Strict "Wasted Round" Rule:** The prompt required that "Invalid input wastes the round". I had to choose between a friendly UX (asking "did you mean rock?") or strict compliance. I chose **strict compliance**. If a user types "Lizard", the logic engine catches it, increments the round counter, and awards zero points. This felt more aligned with a "Referee" persona.


* **Handling API Limits:** During testing, I hit the Google Free Tier rate limit (429 Errors). Instead of letting the program crash, I added a **Failover Mechanism**. If the API is busy, the game catches the error and switches to a "Backup Referee" mode that prints the raw engine result. This keeps the user's session alive even if the cloud service hiccups.

## 4. Improvements with More Time

* **Persistence:** Currently, the state is in-memory. If I had more time, I would implement a simple SQLite database so a user could close the script and resume their game later.
* **Better UI:** Moving from the CLI to a web interface (like Streamlit) would make the "Round History" much easier to visualize.

## Step 1: Imports & State Schema Definition

# **Core Domain Model**



In [ ]:
import json
import random
from enum import Enum
from dataclasses import dataclass, field, asdict
from typing import List, Optional, Dict

# Enums help avoid hard-coded strings scattered across the codebase.
# This reduces bugs caused by typos and makes the game logic easier to reason about.
class Move(Enum):
    ROCK = "rock"
    PAPER = "paper"
    SCISSORS = "scissors"
    BOMB = "bomb"
    INVALID = "invalid" # Used when the user input doesn’t match any valid move.
                        # An invalid move still consumes the round, as per the rules.

class GameStatus(Enum): # The game is still running and accepting moves
    ACTIVE = "active"
    FINISHED = "finished" # The game has completed all rounds and should no longer accept input

@dataclass
class GameState:
    """
    The Single Source of Truth for the game.
    This object holds the entire context required to resolve a turn.
    """
    round_count: int = 1 # Tracks the current round number (1-indexed for clarity)
    max_rounds: int = 3  # Maximum number of rounds allowed in a single game
    user_score: int = 0  # Score counters for the user and the bot
    bot_score: int = 0

    # Flags to enforce the “bomb can be used only once per game” rule
    user_bomb_used: bool = False
    bot_bomb_used: bool = False

    status: GameStatus = GameStatus.ACTIVE # Indicates whether the game is still active or already finished

    # Stores human-readable summaries of each round
    # This helps with clear, round-by-round feedback to the user
    match_history: List[str] = field(default_factory=list)

    def to_json(self) -> str:
        """Helper to serialize state for the Agent/Tool interface."""
        data = asdict(self)
        # Convert Enum values to strings so the JSON stays clean and readable
        data['status'] = self.status.value
        return json.dumps(data, indent=2)

## Step 2: The Game Engine Class

# **Logic Engine**

In [31]:

class GameEngine:
    def __init__(self):
        # I have initialized a fresh state for a new game
        self.state = GameState()

    def _get_bot_move(self) -> Move:
        """
        Determines the bot's move.
        Strategy:
        1. If it's the final round (3), we are losing/tied, and have a Bomb -> USE IT.
        2. Otherwise, pick randomly from available valid moves.
        """
        # Intelligent Heuristic: Desperate times call for desperate measures
        is_final_round = self.state.round_count == self.state.max_rounds
        is_desperate = self.state.bot_score <= self.state.user_score

        if is_final_round and is_desperate and not self.state.bot_bomb_used:
            return Move.BOMB

        # Standard pool of moves
        choices = [Move.ROCK, Move.PAPER, Move.SCISSORS]
        if not self.state.bot_bomb_used:
            choices.append(Move.BOMB)

        return random.choice(choices)

    def _determine_winner(self, user: Move, bot: Move) -> str:
        """
        Pure logic to compare two moves.
        Returns: 'user', 'bot', or 'draw'
        """
        # 1. Handle Invalid/Wasted rounds immediately
        if user == Move.INVALID:
            return "bot" # Techincally user wasted turn, but we can treat as a loss or just no-score.
                         # Guidelines say "wastes the round". We'll handle score update in resolve_round.

        if user == bot:
            return "draw"

        # 2. Bomb Logic
        if user == Move.BOMB:
            return "user" # Bomb beats everything (except bomb, handled in draw)
        if bot == Move.BOMB:
            return "bot"

        # 3. Standard RPS Logic
        wins = {
            Move.ROCK: Move.SCISSORS,
            Move.PAPER: Move.ROCK,
            Move.SCISSORS: Move.PAPER
        }

        if wins[user] == bot:
            return "user"
        return "bot"

    def resolve_round(self, user_input_str: str) -> str:
      # The resolve_round method in the GameEngine class is explicitly defined and passed as a tool (tools_list = [referee.resolve_round]) to the GenerativeModel.
        """
        The PUBLIC method the Agent will call.
        Takes raw string -> Updates State -> Returns JSON result.
        """
        if self.state.status == GameStatus.FINISHED:
            return json.dumps({"error": "Game is already over. Reset to play again."})

        # --- A. Parse & Validate User Input ---
        # Normalize input to lowercase for robustness
        clean_input = user_input_str.lower().strip()
        user_move = Move.INVALID

        # Map string to Enum
        try:
            # Check if it's a valid enum value
            matched_move = Move(clean_input)

            # SPECIAL CHECK: Bomb Rule
            if matched_move == Move.BOMB and self.state.user_bomb_used:
                # If user try to bomb twice time then get invalid move as punishment
                user_move = Move.INVALID
            else:
                user_move = matched_move

        except ValueError:
            # If input is "Spock" or gibberish, then it throws invalid
            user_move = Move.INVALID

        # --- B. Execute Bot Strategy ---
        bot_move = self._get_bot_move()

        # --- C. Update State (Bomb Usage) ---
        if user_move == Move.BOMB:
            self.state.user_bomb_used = True
        if bot_move == Move.BOMB:
            self.state.bot_bomb_used = True

        # --- D. Resolve Outcome ---
        round_winner = "none"
        round_log = ""

        if user_move == Move.INVALID:
            # Requirement: "Invalid input wastes the round"
            round_log = f"Round {self.state.round_count}: Invalid input '{clean_input}'. Round wasted!"
            # No points awarded.
        else:
            winner_key = self._determine_winner(user_move, bot_move)

            if winner_key == "user":
                self.state.user_score += 1
                round_winner = "User"
            elif winner_key == "bot":
                self.state.bot_score += 1
                round_winner = "Bot"
            else:
                round_winner = "Draw"

            round_log = f"Round {self.state.round_count}: You played {user_move.value}, Bot played {bot_move.value}. Winner: {round_winner}."

        # Update History
        self.state.match_history.append(round_log)

        # --- E. Check Game Over ---
        # Increment round
        self.state.round_count += 1

        # Check if the game ends
        # Logic: If round > 3, we are done.
        if self.state.round_count > self.state.max_rounds:
            self.state.status = GameStatus.FINISHED
            final_result = "DRAW"
            if self.state.user_score > self.state.bot_score:
                final_result = "USER WINS GAME"
            elif self.state.bot_score > self.state.user_score:
                final_result = "BOT WINS GAME"

            # Add final summary to history
            self.state.match_history.append(f"GAME OVER. Final Score: You {self.state.user_score} - {self.state.bot_score} Bot. Result: {final_result}")


        # Full state has been returned as JSON file so Agent can read
        return self.state.to_json()

## Step 3: Agent Setup & Main Loop

# **AGENT CONFIGURATION AND EXECUTION**

In [ ]:
import os
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool
import time # Import the time module for sleep functionality
from google.api_core.exceptions import TooManyRequests # Import specific exception for rate limits

def run_game(api_key_str: str):
    """
    Main entry point. Sets up the agent, binds the tool, and runs the loop.
    Args:
        api_key_str: Your Google Gemini API Key.
    """

    # 1. Configure the Google GenAI SDK
    # For ensuring that our agent can authenticate, we have made it explicitly
    try:
        genai.configure(api_key=api_key_str)
    except Exception as e:
        print(f"Error configuring API: {e}")
        return

    # 2. Setup the Game Engine (Our Logic Kernel)
    # Here single source of truth appears
    referee = GameEngine()

    # 3. Define the Tool Wrapper
    # We add the 'resolve_round' method. The SDK converts this python function into a tool schema automatically.
    tools_list = [referee.resolve_round]

    # 4. System Prompt
    # A strict system instruction has been given to force the model to use the tool.
    system_instruction = """
    You are the Official Referee for a Rock-Paper-Scissors-Plus game.

    YOUR PROTOCOL:
    1. RECEIVE move from user.
    2. CALL the `resolve_round` tool with the user's move immediately.
    3. READ the JSON output from the tool.
    4. REPORT the result to the user clearly.

    CRITICAL RULES:
    - DO NOT decide the winner yourself. Only report what the tool says.
    - If the tool says "Round wasted", explain that the input was invalid.
    - Keep output under 5 lines.
    - If the tool says "GAME OVER", announce the final score and say goodbye.
    """

    # 5. Initialize the Model
    # We use 'gemini-2.5-flash' as it is generally available and stable.
    model = genai.GenerativeModel(
        model_name='gemini-2.5-flash',
        tools=tools_list,
        system_instruction=system_instruction
    )

    # 6. Start the Chat Session
    # 'enable_automatic_function_calling' handles the Tool-Use loop for us.
    chat = model.start_chat(enable_automatic_function_calling=True)

    print("--- ROCK-PAPER-SCISSORS-PLUS REFEREE ONLINE ---")
    print("Rules: Best of 3. Valid moves: Rock, Paper, Scissors, Bomb (1x/game).")
    print("Type 'exit' to quit.\n")

    # ------------------------------------------------------------------
    # 4. THE EVENT LOOP
    # ------------------------------------------------------------------
    while True:
        # A. Check State Barrier
        # We check the Engine state directly. If the game is done, we stop.
        # This guarantees we never play a 4th round, satisfying the guidelines strictly.
        if referee.state.status == GameStatus.FINISHED:
            print("\n--- MATCH CONCLUDED ---")
            # We print the final history log from the state for verification
            print(referee.state.match_history[-1])
            break

        # B. User Input Layer
        try:
            user_input = input(f"Round {referee.state.round_count} > ").strip()
        except (KeyboardInterrupt, EOFError):
            print("\nGame aborted by user.")
            break

        if user_input.lower() in ['exit', 'quit']:
            print("Exiting game.")
            break

        # C. Agent Execution Layer
        try:
            # The agent will now automatically:
            # 1. Call resolve_round(user_input)
            # 2. Get the new JSON state
            # 3. Formulate a text response
            response = chat.send_message(user_input)

            # Introduce a delay to mitigate API rate limits
            time.sleep(2)

            # Print the Agent's commentary
            print(f"Referee: {response.text}")

        except TooManyRequests as e: # Catch TooManyRequests specifically
            # Specific handling for API rate limits (429 Resource Exhausted)
            print("Referee: API rate limit hit! Switching to Fallback Referee mode.")
            # Use the last state from the engine directly as a fallback
            if referee.state.match_history:
                print(f"Fallback Referee: {referee.state.match_history[-1]}")
            else:
                print("Fallback Referee: No history available to report.")
        except Exception as e:
            # Graceful error handling (Satisfies "No crashes") for other unexpected errors
            print(f"System Error: An unexpected error occurred: {e}")
            break

if __name__ == "__main__":
    # --- CONFIGURATION ---
    # Here in "GEMINI_API_KEY" Real-time gemini api key should be given and publicly not should be revealed.
    MY_API_KEY = "GEMINI_API_KEY" # Gemini API key is part of Google’s LLM stack that ADK is built for.

    if MY_API_KEY == "GEMINI_API_KEY": # If key is not given
        print("Error: Please set your valid Google API Key in the `MY_API_KEY` variable within the code.")
    else:
        run_game(MY_API_KEY)

--- ROCK-PAPER-SCISSORS-PLUS REFEREE ONLINE ---
Rules: Best of 3. Valid moves: Rock, Paper, Scissors, Bomb (1x/game).
Type 'exit' to quit.

Round 1 > Rock
Referee: Round 2: You played rock, Bot played bomb. Winner: Bot.
Bot score: 1. Your score: 0.
Round 2 > Scissors
Referee: Round 2: You played scissors, Bot played scissors. Winner: Draw.
Bot score: 1. Your score: 0.
Round 3 > Bomb
Referee: GAME OVER. Final Score: You 1 - 1 Bot. Result: DRAW. Goodbye!

--- MATCH CONCLUDED ---
GAME OVER. Final Score: You 1 - 1 Bot. Result: DRAW
